Классификация


Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
from collections import Counter

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier


Загрузка и первичный анализ данных

In [2]:
df_cancer = pd.read_csv('cancer-risk-factors.csv')
print("Размер датасета:", df_cancer.shape)
print("\nПервые 5 строк:")
print(df_cancer.head())

Размер датасета: (2000, 21)

Первые 5 строк:
  Patient_ID Cancer_Type  Age  Gender  Smoking  Alcohol_Use  Obesity  \
0     LU0000      Breast   68       0        7            2        8   
1     LU0001    Prostate   74       1        8            9        8   
2     LU0002        Skin   55       1        7           10        7   
3     LU0003       Colon   61       0        6            2        2   
4     LU0004        Lung   67       1       10            7        4   

   Family_History  Diet_Red_Meat  Diet_Salted_Processed  ...  \
0               0              5                      3  ...   
1               0              0                      3  ...   
2               0              3                      3  ...   
3               0              6                      2  ...   
4               0              6                      3  ...   

   Physical_Activity  Air_Pollution  Occupational_Hazards  BRCA_Mutation  \
0                  4              6                     3    

Удаляем столбцы

In [3]:
columns_to_drop = ['Overall_Risk_Score', 'Risk_Level', 'Patient_ID']
df_cancer = df_cancer.drop(columns=columns_to_drop)
print("\nСтолбцы после удаления:")
print(df_cancer.columns.tolist())


Столбцы после удаления:
['Cancer_Type', 'Age', 'Gender', 'Smoking', 'Alcohol_Use', 'Obesity', 'Family_History', 'Diet_Red_Meat', 'Diet_Salted_Processed', 'Fruit_Veg_Intake', 'Physical_Activity', 'Air_Pollution', 'Occupational_Hazards', 'BRCA_Mutation', 'H_Pylori_Infection', 'Calcium_Intake', 'BMI', 'Physical_Activity_Level']


Разделяем признаки и целевую переменную

In [4]:
y = df_cancer['Cancer_Type']
X = df_cancer.drop(columns=['Cancer_Type'])

print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (2000, 17)
y shape: (2000,)


Кодируем целевую переменную

In [5]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print("Классы рака:", label_encoder.classes_)


Классы рака: ['Breast' 'Colon' 'Lung' 'Prostate' 'Skin']


Разделяем данные

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y_encoded,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded
)


Препроцессинг признаков

In [7]:
numeric_features = X.select_dtypes(include=[np.number]).columns
categorical_features = X.select_dtypes(exclude=[np.number]).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), numeric_features),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_features)
    ]
)


Обучаем Random Forest

In [8]:
rf_baseline = Pipeline([
    ('preprocessor', preprocessor),
    ('rf', RandomForestClassifier(
        n_estimators=100,
        random_state=42
    ))
])

rf_baseline.fit(X_train, y_train)



,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('rf', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers conta

Предсказание

In [9]:
y_pred_baseline = rf_baseline.predict(X_test)


Оценка качества

In [10]:
print("Baseline Accuracy:", accuracy_score(y_test, y_pred_baseline))

print("\nBaseline Classification Report:")
print(classification_report(
    y_test,
    y_pred_baseline,
    target_names=label_encoder.classes_
))


Baseline Accuracy: 0.7825

Baseline Classification Report:
              precision    recall  f1-score   support

      Breast       0.82      0.82      0.82        92
       Colon       0.80      0.79      0.79        84
        Lung       0.79      0.91      0.85       105
    Prostate       0.75      0.75      0.75        61
        Skin       0.71      0.52      0.60        58

    accuracy                           0.78       400
   macro avg       0.77      0.76      0.76       400
weighted avg       0.78      0.78      0.78       400



Улучшение бейзлайна. 

Гипотезы

Гипотеза 1
Увеличение числа деревьев улучшит стабильность ансамбля

Гипотеза 2
Ограничение глубины уменьшит переобучение

Гипотеза 3
Подбор min_samples_split и min_samples_leaf улучшит обобщение.

Подбор гиперпараметров (GridSearch)

In [11]:
pipe_rf = Pipeline([
    ('preprocessor', preprocessor),
    ('rf', RandomForestClassifier(random_state=42))
])

param_grid = {
    'rf__n_estimators': [100, 200],
    'rf__max_depth': [None, 5, 10],
    'rf__min_samples_split': [2, 5, 10],
    'rf__min_samples_leaf': [1, 3, 5]
}

grid_rf = GridSearchCV(
    pipe_rf,
    param_grid,
    cv=5,
    scoring='f1_weighted',
    n_jobs=-1
)

grid_rf.fit(X_train, y_train)

print("Лучшие параметры:", grid_rf.best_params_)


Лучшие параметры: {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 5, 'rf__n_estimators': 200}


Модель с улучшенным бейзлайном

In [13]:
best_rf = grid_rf.best_estimator_

y_pred_improved = best_rf.predict(X_test)

Оценка качества

In [14]:
print("Improved Accuracy:", accuracy_score(y_test, y_pred_improved))

print("\nImproved Classification Report:")
print(classification_report(
    y_test,
    y_pred_improved,
    target_names=label_encoder.classes_
))

Improved Accuracy: 0.775

Improved Classification Report:
              precision    recall  f1-score   support

      Breast       0.82      0.82      0.82        92
       Colon       0.78      0.76      0.77        84
        Lung       0.80      0.90      0.85       105
    Prostate       0.74      0.75      0.75        61
        Skin       0.65      0.52      0.58        58

    accuracy                           0.78       400
   macro avg       0.76      0.75      0.75       400
weighted avg       0.77      0.78      0.77       400



Было

In [15]:
print("Baseline Accuracy:", accuracy_score(y_test, y_pred_baseline))

print("\nBaseline Classification Report:")
print(classification_report(
    y_test,
    y_pred_baseline,
    target_names=label_encoder.classes_
))


Baseline Accuracy: 0.7825

Baseline Classification Report:
              precision    recall  f1-score   support

      Breast       0.82      0.82      0.82        92
       Colon       0.80      0.79      0.79        84
        Lung       0.79      0.91      0.85       105
    Prostate       0.75      0.75      0.75        61
        Skin       0.71      0.52      0.60        58

    accuracy                           0.78       400
   macro avg       0.77      0.76      0.76       400
weighted avg       0.78      0.78      0.78       400



Выводы: После проверки гипотез улучшения не было, гипотезы не подтвердились.

Имплементация алгоритма машинного обучения


In [22]:
class MyDecisionTreeClassifier:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y, depth=0)

    def _build_tree(self, X, y, depth):
        # стоп-условия
        if (
            len(y) == 0 or
            len(set(y)) == 1 or
            (self.max_depth is not None and depth >= self.max_depth)
        ):
            return Counter(y).most_common(1)[0][0]

        feature = np.random.randint(X.shape[1])
        threshold = np.median(X[:, feature])

        left_mask = X[:, feature] <= threshold
        right_mask = X[:, feature] > threshold

        # плохое разбиение — останавливаемся
        if left_mask.sum() == 0 or right_mask.sum() == 0:
            return Counter(y).most_common(1)[0][0]

        return {
            'feature': feature,
            'threshold': threshold,
            'left': self._build_tree(X[left_mask], y[left_mask], depth + 1),
            'right': self._build_tree(X[right_mask], y[right_mask], depth + 1)
        }

    def _predict_one(self, x, node):
        if not isinstance(node, dict):
            return node
        if x[node['feature']] <= node['threshold']:
            return self._predict_one(x, node['left'])
        else:
            return self._predict_one(x, node['right'])

    def predict(self, X):
        return np.array([self._predict_one(x, self.tree) for x in X])


class MyRandomForestClassifier:
    def __init__(self, n_estimators=10, max_depth=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        n_samples = X.shape[0]

        for _ in range(self.n_estimators):
            idx = np.random.choice(n_samples, n_samples, replace=True)
            tree = MyDecisionTreeClassifier(max_depth=self.max_depth)
            tree.fit(X[idx], y[idx])
            self.trees.append(tree)

    def predict(self, X):
        preds = np.array([tree.predict(X) for tree in self.trees])
        return np.array([
            Counter(preds[:, i]).most_common(1)[0][0]
            for i in range(X.shape[0])
        ])


Обучение и подготовка данных

In [23]:
X_num = df_cancer[numeric_features]
y_num = y_encoded

X_num = SimpleImputer(strategy='mean').fit_transform(X_num)

X_train_n, X_test_n, y_train_n, y_test_n = train_test_split(
    X_num, y_num, test_size=0.2, random_state=42, stratify=y_num
)


mmy_rf_base = MyRandomForestClassifier(
    n_estimators=10,
    max_depth=5
)

my_rf_base.fit(X_train_n, y_train_n)

y_pred_my_base = my_rf_base.predict(X_test_n)

Оценка качества

In [27]:
print("My RF Baseline Accuracy:", accuracy_score(y_test_n, y_pred_my_base))

print("\nMy RF Classification Report:")
print(classification_report(
    y_test_n,
    y_pred_my_base,
    target_names=label_encoder.classes_
))


My RF Baseline Accuracy: 0.525

My RF Classification Report:
              precision    recall  f1-score   support

      Breast       0.60      0.61      0.60        92
       Colon       0.50      0.61      0.55        84
        Lung       0.57      0.73      0.64       105
    Prostate       0.40      0.31      0.35        61
        Skin       0.33      0.12      0.18        58

    accuracy                           0.53       400
   macro avg       0.48      0.48      0.46       400
weighted avg       0.50      0.53      0.50       400



Было у алгоритма из sklearn


In [25]:
print("Baseline Accuracy:", accuracy_score(y_test, y_pred_baseline))

print("\nBaseline Classification Report:")
print(classification_report(
    y_test,
    y_pred_baseline,
    target_names=label_encoder.classes_
))


Baseline Accuracy: 0.7825

Baseline Classification Report:
              precision    recall  f1-score   support

      Breast       0.82      0.82      0.82        92
       Colon       0.80      0.79      0.79        84
        Lung       0.79      0.91      0.85       105
    Prostate       0.75      0.75      0.75        61
        Skin       0.71      0.52      0.60        58

    accuracy                           0.78       400
   macro avg       0.77      0.76      0.76       400
weighted avg       0.78      0.78      0.78       400



Выводы: собственный алгоритм показал метрики хуже  бейзлана из sklearn. Это обьясняется упрощенной реализацией алгоритма.

Улучшение алгоритма

Гипотезы

Увеличение числа деревьев улучшит устойчивость

Ограничение глубины уменьшит переобучение

In [28]:
my_rf_improved = MyRandomForestClassifier(
    n_estimators=50,
    max_depth=6
)

my_rf_improved.fit(X_train_n, y_train_n)

y_pred_my_imp = my_rf_improved.predict(X_test_n)



Оценка


In [31]:
print("My RF Improved Accuracy:", accuracy_score(y_test_n, y_pred_my_imp))

print("My RF Improved Classification Report:")
print(classification_report(
    y_test,
    y_pred_baseline,
    target_names=label_encoder.classes_
))


My RF Improved Accuracy: 0.535
My RF Improved Classification Report:
              precision    recall  f1-score   support

      Breast       0.82      0.82      0.82        92
       Colon       0.80      0.79      0.79        84
        Lung       0.79      0.91      0.85       105
    Prostate       0.75      0.75      0.75        61
        Skin       0.71      0.52      0.60        58

    accuracy                           0.78       400
   macro avg       0.77      0.76      0.76       400
weighted avg       0.78      0.78      0.78       400



Было у улучшенного бейзлайна из sklearn

In [32]:
print("Improved Accuracy:", accuracy_score(y_test, y_pred_improved))

print("\nImproved Classification Report:")
print(classification_report(
    y_test,
    y_pred_improved,
    target_names=label_encoder.classes_
))

Improved Accuracy: 0.775

Improved Classification Report:
              precision    recall  f1-score   support

      Breast       0.82      0.82      0.82        92
       Colon       0.78      0.76      0.77        84
        Lung       0.80      0.90      0.85       105
    Prostate       0.74      0.75      0.75        61
        Skin       0.65      0.52      0.58        58

    accuracy                           0.78       400
   macro avg       0.76      0.75      0.75       400
weighted avg       0.77      0.78      0.77       400



Выводы: собственный алгоритм показал метрики хуже  бейзлана из sklearn. Это обьясняется упрощенной реализацией алгоритма.

Регрессия


Загрузка и первичный анализ данных

In [33]:
df_productivity = pd.read_csv('social_media_vs_productivity.csv')
print("Размер датасета:", df_productivity.shape)
print("\nПервые 5 строк:")
print(df_productivity.head())

Размер датасета: (30000, 19)

Первые 5 строк:
   age  gender    job_type  daily_social_media_time  \
0   56    Male  Unemployed                 4.180940   
1   46    Male      Health                 3.249603   
2   32    Male     Finance                      NaN   
3   60  Female  Unemployed                      NaN   
4   25    Male          IT                      NaN   

  social_platform_preference  number_of_notifications  work_hours_per_day  \
0                   Facebook                       61            6.753558   
1                    Twitter                       59            9.169296   
2                    Twitter                       57            7.910952   
3                   Facebook                       59            6.355027   
4                   Telegram                       66            6.214096   

   perceived_productivity_score  actual_productivity_score  stress_level  \
0                      8.040464                   7.291555           4.0   
1       

Обработка пропусков и уменьшения размера датасета


In [34]:
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score

df_productivity_clean = df_productivity.dropna(
    subset=['actual_productivity_score']
)

df_productivity_clean = df_productivity_clean.iloc[:1000].copy()




Разделяем признаки и целевую переменную

In [35]:
target = 'actual_productivity_score'

X = df_productivity_clean.drop(columns=[target])
y = df_productivity_clean[target]


Препроцессинг



In [36]:
numeric_features = X.select_dtypes(include=[np.number]).columns
categorical_features = X.select_dtypes(exclude=[np.number]).columns


from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), numeric_features),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_features)
    ]
)


Train / Test split

In [37]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


Baseline RandomForest 

In [39]:
from sklearn.ensemble import RandomForestRegressor

rf_baseline = Pipeline([
    ('preprocessor', preprocessor),
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42))
])

rf_baseline.fit(X_train, y_train)
y_pred_baseline = rf_baseline.predict(X_test)

Оценка качества

In [41]:
print("Baseline MSE:", mean_squared_error(y_test, y_pred_baseline))
print("Baseline R2:", r2_score(y_test, y_pred_baseline))

Baseline MSE: 0.2737884532538143
Baseline R2: 0.9189617388181652


Улучшение бейзлайна

Гипотезы:
Масштабирование численных признаков (для стабильности)
Подбор количества деревьев и глубины
Ограничение min_samples_split для устойчивости

In [ ]:

from sklearn.preprocessing import StandardScaler, OneHotEncoder

pipe_improved = Pipeline([
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler(with_mean=False)),  
    ('rf', RandomForestRegressor(random_state=42))
])

Подбор гиперпараметров

In [48]:
param_grid = {
    'rf__n_estimators': [50, 100],
    'rf__max_depth': [None, 5, 10],
    'rf__min_samples_split': [2, 5, 10],
    'rf__max_features': ['sqrt', 'log2']
}

grid_rf = GridSearchCV(
    pipe_improved,
    param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    error_score='raise'
)

grid_rf.fit(X_train, y_train)

print("Лучшие параметры:", grid_rf.best_params_)

Лучшие параметры: {'rf__max_depth': None, 'rf__max_features': 'sqrt', 'rf__min_samples_split': 2, 'rf__n_estimators': 50}


Оценка улучшенного бейзлайна

In [49]:
best_rf = grid_rf.best_estimator_
y_pred_improved = best_rf.predict(X_test)

print("Improved MSE:", mean_squared_error(y_test, y_pred_improved))
print("Improved R2:", r2_score(y_test, y_pred_improved))



Improved MSE: 0.5175954566189371
Improved R2: 0.8467976450375302


Было

In [50]:
print("Baseline MSE:", mean_squared_error(y_test, y_pred_baseline))
print("Baseline R2:", r2_score(y_test, y_pred_baseline))

Baseline MSE: 0.2737884532538143
Baseline R2: 0.9189617388181652


Выводы: улучшенный бейзлайн Random Tree стал значительно хуже. Гипотезы не подтвердились.

Собственная реализация Random Tree

In [51]:
from collections import Counter


class MyDecisionTreeRegressor:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y, depth=0)

    def _build_tree(self, X, y, depth):
        if len(y) == 0:
            return 0
        if len(set(y)) == 1 or (self.max_depth is not None and depth >= self.max_depth):
            return np.mean(y)

        feature = np.random.randint(X.shape[1])
        threshold = np.median(X[:, feature])

        left_mask = X[:, feature] <= threshold
        right_mask = X[:, feature] > threshold

        if left_mask.sum() == 0 or right_mask.sum() == 0:
            return np.mean(y)

        return {
            'feature': feature,
            'threshold': threshold,
            'left': self._build_tree(X[left_mask], y[left_mask], depth + 1),
            'right': self._build_tree(X[right_mask], y[right_mask], depth + 1)
        }

    def _predict_one(self, x, node):
        if not isinstance(node, dict):
            return node
        if x[node['feature']] <= node['threshold']:
            return self._predict_one(x, node['left'])
        else:
            return self._predict_one(x, node['right'])

    def predict(self, X):
        return np.array([self._predict_one(x, self.tree) for x in X])


class MyRandomForestRegressor:
    def __init__(self, n_estimators=10, max_depth=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        n_samples = X.shape[0]

        for _ in range(self.n_estimators):
            # Bootstrap выборка
            idx = np.random.choice(n_samples, n_samples, replace=True)
            tree = MyDecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X[idx], y[idx])
            self.trees.append(tree)

    def predict(self, X):
        preds = np.array([tree.predict(X) for tree in self.trees])
        return np.mean(preds, axis=0)


Подготовка данных  (используем только числовые признаки)

In [52]:
X_num = df_productivity_clean[numeric_features].values
y_num = df_productivity_clean[target].values

X_num = SimpleImputer(strategy='mean').fit_transform(X_num)

X_train_n, X_test_n, y_train_n, y_test_n = train_test_split(
    X_num, y_num, test_size=0.2, random_state=42
)


Обучение и оценка

In [53]:
my_rf_base = MyRandomForestRegressor(n_estimators=10, max_depth=None)
my_rf_base.fit(X_train_n, y_train_n)
y_pred_my_base = my_rf_base.predict(X_test_n)

print("My RF Baseline MSE:", mean_squared_error(y_test_n, y_pred_my_base))
print("My RF Baseline R2:", r2_score(y_test_n, y_pred_my_base))


My RF Baseline MSE: 1.4455464412472652
My RF Baseline R2: 0.5721347315269383


Было у алгоритма из sklearn

In [54]:
print("Baseline MSE:", mean_squared_error(y_test, y_pred_baseline))
print("Baseline R2:", r2_score(y_test, y_pred_baseline))

Baseline MSE: 0.2737884532538143
Baseline R2: 0.9189617388181652


Вывод: собственная реализация оказалась значительно хуже. Это обьясняется упрощенной реализацией.


Используем гипотезы для собственного алгоритма(ограничение глубины уменьшит переобучение, увеличить число деревьев (n_estimators) стабилизирует предсказания).

In [55]:
my_rf_improved = MyRandomForestRegressor(n_estimators=50, max_depth=8)
my_rf_improved.fit(X_train_n, y_train_n)
y_pred_my_improved = my_rf_improved.predict(X_test_n)

print("My RF Improved MSE:", mean_squared_error(y_test_n, y_pred_my_improved))
print("My RF Improved R2:", r2_score(y_test_n, y_pred_my_improved))


My RF Improved MSE: 1.3056813058635777
My RF Improved R2: 0.6135332172437497


Было у улучшенного бейзлайна из sklearn

In [56]:
best_rf = grid_rf.best_estimator_
y_pred_improved = best_rf.predict(X_test)

print("Improved MSE:", mean_squared_error(y_test, y_pred_improved))
print("Improved R2:", r2_score(y_test, y_pred_improved))



Improved MSE: 0.5175954566189371
Improved R2: 0.8467976450375302


Вывод: Результаты у улучшенного собственного алгоритма оказались  хуже улучшенного алгоритма из sklearn, что обьясняется упрощенностью реализации.